In [2]:
from pennylane import numpy as np
import pennylane as qml


In [3]:
def angstrom_to_bohr(dist):
    return dist*1.88973


In [4]:
#symbols = ["H", "H"]
#coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

#H, n_qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, grouping_method='qwc')
#print("Number of qubits = ", n_qubits)
#print("The Hamiltonian is ", H)

In [5]:

n_electrons = 4
n_orbitals = 6

symbols = ["H", "Be", "H"]
coordinates = angstrom_to_bohr(np.array([0.0, 0.0, -1.291, 0.0, 0.0,0.0, 0.0, 0.0, 1.291]))
#coordinates = angstrom_to_bohr(np.array([0.0, 0.0, -1.33, 0.0, 0.0,0.0, 0.0, 0.0, 1.33]))


H, n_qubits = qml.qchem.molecular_hamiltonian(symbols,
                                                coordinates,
                                                active_electrons=n_electrons,
                                                active_orbitals=n_orbitals,
                                                grouping_method='qwc')
print("Number of qubits = ", n_qubits)
print("The Hamiltonian is ", H)

Number of qubits =  12
The Hamiltonian is    (-12.502075969687327) [I0]
+ (-0.7841382553568715) [Z11]
+ (-0.7841382553568713) [Z10]
+ (-0.5067882208317611) [Z9]
+ (-0.506788220831761) [Z8]
+ (-0.4272291179703887) [Z6]
+ (-0.4272291179703886) [Z7]
+ (-0.4272291179701972) [Z4]
+ (-0.4272291179701972) [Z5]
+ (-0.2518493116366934) [Z3]
+ (-0.2518493116366931) [Z2]
+ (-0.23090165792730682) [Z1]
+ (-0.2309016579273067) [Z0]
+ (0.06282016880305943) [Z0 Z2]
+ (0.06282016880305943) [Z1 Z3]
+ (0.07473679688924734) [Z8 Z10]
+ (0.07473679688924734) [Z9 Z11]
+ (0.07967196615626496) [Z4 Z8]
+ (0.07967196615626496) [Z5 Z9]
+ (0.0796719661563092) [Z6 Z8]
+ (0.0796719661563092) [Z7 Z9]
+ (0.08056539538945896) [Z0 Z4]
+ (0.08056539538945896) [Z1 Z5]
+ (0.08056539538950375) [Z0 Z6]
+ (0.08056539538950375) [Z1 Z7]
+ (0.08094294505965398) [Z0 Z8]
+ (0.08094294505965398) [Z1 Z9]
+ (0.08210113323407828) [Z2 Z8]
+ (0.08210113323407828) [Z3 Z9]
+ (0.08629160422428994) [Z2 Z4]
+ (0.08629160422428994) [Z3 Z5]
+ 

In [6]:
print(len(H.coeffs))

327


In [7]:
generators = qml.symmetry_generators(H)
paulixops = qml.paulix_ops(generators, n_qubits)
paulix_sector = qml.qchem.optimal_sector(H, generators, n_electrons)

/Users/Brian/workspace/Research/QHack2023/.conda/lib/python3.10/site-packages/pennylane/operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/Users/Brian/workspace/Research/QHack2023/.conda/lib/python3.10/site-packages/pennylane/operation.py:1993: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


In [8]:
H_tapered = qml.taper(H, generators, paulixops, paulix_sector)
print(H_tapered)

  ((-12.277146424567801+0j)) [I0]
+ ((-0.8544582359407763+0j)) [Z6]
+ ((-0.8544582359403944+0j)) [Z4]
+ ((-0.5067882208317603+0j)) [Z8]
+ ((-0.2518493116366933+0j)) [Z3]
+ ((-0.2518493116366931+0j)) [Z2]
+ ((-0.23090165792730663+0j)) [Z1]
+ ((-0.2309016579273066+0j)) [Z0]
+ ((0.0041076977364904455+0j)) [X4]
+ ((0.004107697736492724+0j)) [X6]
+ ((-0.04079935681017314+0j)) [Y2 Y3]
+ ((-0.014134246771489997+0j)) [Y0 Y1]
+ ((-0.0027407367227268316+0j)) [Y1 Y3]
+ ((0.012124691336499021+0j)) [Y4 Y6]
+ ((0.012124691336499021+0j)) [X4 X6]
+ ((0.012760887739289128+0j)) [X4 X8]
+ ((0.012760887739289128+0j)) [Y4 Y8]
+ ((0.012760887739296229+0j)) [X6 X8]
+ ((0.012760887739296229+0j)) [Y6 Y8]
+ ((0.014336058712826286+0j)) [Y1 Y8]
+ ((0.018021129271588877+0j)) [Y1 Y2]
+ ((0.024084986684441062+0j)) [Y3 Y8]
+ ((0.06282016880305939+0j)) [Z0 Z2]
+ ((0.06282016880305939+0j)) [Z1 Z3]
+ ((0.08210113323407821+0j)) [Z2 Z8]
+ ((0.1029062553739197+0j)) [Z3 Z8]
+ ((0.10411863541541243+0j)) [Z0 Z3]
+ ((0.1041186

In [9]:
print(len(H_tapered.ops))

268


In [10]:
state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
                                   num_electrons=n_electrons, num_wires=len(H.wires))
print(state_tapered)

[1 1 1 1 0 0 0]


In [11]:
dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def circuit():
    qml.BasisState(np.array(state_tapered), wires=H_tapered.wires)
    return qml.state()

qubit_state = circuit()
HF_energy = qubit_state.T @ qml.utils.sparse_hamiltonian(H_tapered).toarray() @ qubit_state
print(f"HF energy (tapered): {np.real(HF_energy):.8f} Ha")

HF energy (tapered): -15.56135260 Ha


In [12]:
singles, doubles = qml.qchem.excitations(n_electrons, len(H.wires))
print('singles', len(singles), 'doubles ', len(doubles))
tapered_doubles = [
    qml.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=double) for double in doubles
]
tapered_singles = [
    qml.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=single) for single in singles
]

dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)

singles 16 doubles  76


In [13]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(doubles) + len(singles), requires_grad=True)

for n in range(1, 41):
    params, energy = optimizer.step_and_cost(tapered_circuit, params)
    if not n % 5:
        print(f"n: {n}, E: {energy:.8f} Ha")

n: 5, E: -15.59249574 Ha
n: 10, E: -15.59394517 Ha
n: 15, E: -15.59405752 Ha
n: 20, E: -15.59407715 Ha
n: 25, E: -15.59408125 Ha
n: 30, E: -15.59408214 Ha
n: 35, E: -15.59408233 Ha
n: 40, E: -15.59408238 Ha


In [14]:
qubit_state = circuit()
HF_energy = qubit_state.T @ qml.utils.sparse_hamiltonian(H_tapered).toarray() @ qubit_state
print(f"HF energy (tapered): {np.real(HF_energy):.8f} Ha")

HF energy (tapered): -15.56135260 Ha


In [15]:
part = qml.data.load("qchem", molname="BeH2", basis="STO-3G", bondlength=1.33,
                      attributes=["molecule", "fci_energy"])[0]
part.molecule

print('FCI energy :', part.fci_energy, ' Ha')

FCI energy : -15.59512681225857  Ha


In [16]:
@qml.qnode(dev)
def taperedState_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.state()

state = taperedState_circuit(params)
round_state = state[0]
print(np.around(state,6))
print(np.max(state))

[-0.00000e+00+0.j  8.02400e-03+0.j  2.17200e-03+0.j  4.22700e-03+0.j
  2.17200e-03+0.j  4.22700e-03-0.j  2.83000e-04+0.j  0.00000e+00+0.j
  0.00000e+00-0.j  3.84000e-04-0.j -1.81000e-04-0.j  0.00000e+00+0.j
 -1.81000e-04-0.j -0.00000e+00+0.j -0.00000e+00+0.j  0.00000e+00-0.j
 -0.00000e+00+0.j -3.84000e-04+0.j  1.81000e-04+0.j -0.00000e+00-0.j
  1.81000e-04+0.j -0.00000e+00-0.j -0.00000e+00-0.j -0.00000e+00+0.j
 -2.66640e-02-0.j -5.58740e-02-0.j -5.40800e-02-0.j  0.00000e+00+0.j
 -5.40810e-02+0.j  0.00000e+00+0.j -0.00000e+00-0.j -0.00000e+00+0.j
  0.00000e+00-0.j -1.39000e-04+0.j  0.00000e+00+0.j  1.70000e-05+0.j
  0.00000e+00+0.j  1.70000e-05-0.j  0.00000e+00+0.j  0.00000e+00+0.j
 -0.00000e+00-0.j -5.85500e-03+0.j  0.00000e+00-0.j -0.00000e+00-0.j
  0.00000e+00-0.j  0.00000e+00+0.j  0.00000e+00-0.j -0.00000e+00+0.j
 -4.03290e-02-0.j  4.61840e-02+0.j -0.00000e+00-0.j  0.00000e+00+0.j
 -0.00000e+00-0.j  0.00000e+00+0.j -0.00000e+00+0.j  0.00000e+00-0.j
 -0.00000e+00+0.j  1.00330e-02+0.j